<center>
<img src="https://upload.wikimedia.org/wikipedia/commons/4/47/Acronimo_y_nombre_uc3m.png"/>

<img src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by-nc-sa.png" width=15%/>
</center>   

# Ajustar un modelo pre-entrenado para la clasifición de textos (Tensorflow)

Como en el notebook anterior, también vamos a ajustar (fine-tuning) un modelo transformer (en particular BERT) a la tarea de clasificación de textos, pero esta vez en lugar de utilizar como framework **Pytorch** vamos a usar **Tensorflow**. 

Fuente:
https://huggingface.co/docs/transformers/training

Comenzamos instalando las dos librerías de HuggingFace: transformers y datasets

In [ ]:
!pip install transformers datasets 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 KB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 88.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 1.8 MB/s eta 0:00:00


## Data

Vamos a utilizar el mismo dataset que utlizamos en el notebook anterior, el dataset **trec** (https://huggingface.co/datasets/trec/viewer/default/train). Este dataset está formado por una colección de preguntas que han sido clasificadas en un función del tipo de respuesta que espera. 
Estas clases son las siguientes (se recogen en el campo coarse_label).

- 'ABBR' (0): la respuesta esperada es una abreviatura.
- 'ENTY' (1): la respuesta esperada es una entidad.
- 'DESC' (2): la respuesta esperada es una descripción.
- 'HUM' (3): la respuesta esperada es una persona.
- 'LOC' (4): la respuesta esperada es un lugar.
- 'NUM' (5): la respuesta esperada es un valor numérico.

El dataset incluye un segundo campo, **fine_label**, donde se da una clasificación más fina del tipo de respuesta esperada para cada pregunta. En este notebook, nos centraremos únicamente en la clasificación basada en coarse_label. 

**Nota:** Una práctica interesante puede ser ajustar el modelo pero ahora para la clasificación basada en las etiquetas de **fine_label**.



In [ ]:
from datasets import load_dataset
dict_dataset = load_dataset("trec")
dict_dataset

Generating train split:   0%|          | 0/5452 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

Dataset trec downloaded and prepared to /root/.cache/huggingface/datasets/trec/default/2.0.0/f2469cab1b5fceec7249fda55360dfdbd92a7a5b545e91ea0f78ad108ffac1c2. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'coarse_label', 'fine_label'],
        num_rows: 5452
    })
    test: Dataset({
        features: ['text', 'coarse_label', 'fine_label'],
        num_rows: 500
    })
})

In [ ]:
TARGET_LABELS = dict_dataset['train'].features['coarse_label'].names
print('Clases: ', TARGET_LABELS)
# eliminamos el campo fine_label y renombramos 'coarse_label' a 'label'
dict_dataset = dict_dataset.remove_columns(['fine_label'])
dict_dataset = dict_dataset.rename_column('coarse_label','label')

dict_dataset

Clases:  ['ABBR', 'ENTY', 'DESC', 'HUM', 'LOC', 'NUM']


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 5452
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 500
    })
})

### Creación del split para validación

Seguimos los mismos pasos que en el ejemplo anterior para crear el split que se utilizará en la fase de entrenamiento para evaluar el modelo en cada epoch:

In [ ]:
aux = dict_dataset['train'].train_test_split(test_size=0.1)
dict_dataset['train']=aux['train']
dict_dataset['val']=aux['test']
del(aux)
dict_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 4906
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 500
    })
    val: Dataset({
        features: ['text', 'label'],
        num_rows: 546
    })
})

### Tokenization
Usamos el mismo tokenizador y la misma función de tokenización


In [ ]:
from transformers import AutoTokenizer
model_name="bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# calculamos MAX_LENGTH
MAX_LENGTH= max([len(tokenizer(text).input_ids) for text in dict_dataset['train']['text']])
print("La longitud máxima de la secuencia es: ", MAX_LENGTH)

MAX_LENGTH = min(512, MAX_LENGTH)
print("max_length", MAX_LENGTH)


def tokenize(example):
    # Keys of the returned dictionary will be added to the dataset as columns
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=MAX_LENGTH)

# aplicamos la tokenización a los tres splits
encoded_dataset = dict_dataset.map(tokenize)
encoded_dataset

La longitud máxima de la secuencia es:  41
max_length 41


Map:   0%|          | 0/4906 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/546 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4906
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 500
    })
    val: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 546
    })
})

## Modelo (usando tensorflow)

Debemos conocer el número de clases. Además, creamos dos diccionarios que nos facilitarán la traducción de identificador a clase de forma muy sencilla. 

In [ ]:
NUM_LABELS = len(TARGET_LABELS)

print('TARGET_LABELS:', TARGET_LABELS, 'NUM_LABELS:', NUM_LABELS )

id2label = {}
for index, label in enumerate(TARGET_LABELS):
    id2label.update({index:label})
    
label2id = {val: key for key, val in id2label.items()}
print(id2label, label2id)


TARGET_LABELS: ['ABBR', 'ENTY', 'DESC', 'HUM', 'LOC', 'NUM'] NUM_LABELS: 6
{0: 'ABBR', 1: 'ENTY', 2: 'DESC', 3: 'HUM', 4: 'LOC', 5: 'NUM'} {'ABBR': 0, 'ENTY': 1, 'DESC': 2, 'HUM': 3, 'LOC': 4, 'NUM': 5}


HuggingFace  proporciona clases preparadas para ajustar un determinado transformer a una tarea de clasificación de textos, que no es otra cosa que la clasificación de una secuencia de tokens. Pytorch cuenta con su conjunto propio de clases, y de la misma forma, HuggingFace también ha desarrollado las clases necesarias para trabajar en el framework de Tensorflow. 
En este caso, la clase concreta es **TFAutoModelForSequenceClassification**. Su atributos, además del nombre del modelo y número de clases, también es necesario pasarle los dos diccionarios que acabamos de crear con la traducción entre clase y su índice: 

In [ ]:
from transformers import TFAutoModelForSequenceClassification # clase de tensorflow para cargar un modelo y prepararlo para la clasificación de textos

model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=NUM_LABELS, id2label=id2label, label2id=label2id )


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


El split de training y validación deben ser pasados al modelo. Para ello debemos utilizar la función **prepare_tf_dataset**, que hara justamente eso, preparar el dataset conforme a tensorflow. Aquí vamos a definir el tamaño del batch, el tokenizador, y también vamos a permitir que las instancias se reordenen aletatoriamente. 

In [ ]:
tf_train_dataset = model.prepare_tf_dataset(
    encoded_dataset["train"],
    shuffle=True,
    batch_size=16,
    tokenizer=tokenizer
)

tf_validation_dataset = model.prepare_tf_dataset(
    encoded_dataset['val'],
    shuffle=False,
    batch_size=16,
    tokenizer=tokenizer,
)
tf_train_dataset

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


<PrefetchDataset element_spec=({'input_ids': TensorSpec(shape=(16, 41), dtype=tf.int64, name=None), 'token_type_ids': TensorSpec(shape=(16, 41), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(16, 41), dtype=tf.int64, name=None)}, TensorSpec(shape=(16,), dtype=tf.int64, name=None))>

ADemás, tenemos que definir algunos hiperparámetros para el modelo, como el número de epochs y el tamaño por lote. Esto nos permite calcular facilmente cuántos lotes se van a ejecutar en cada epoch **batches_pero_epoch**, y el número total de lotes que se van a ejecutar, o lo que es lo mismo, el número total de pasos en el entrenamiento, **total_train_steps** (considerando como paso, cada ejecución de un lote). 

También debemos crear un optimizador, indicando el número total de pasos, y el learning rate. 


In [ ]:
from transformers import create_optimizer
batch_size = 16
num_epochs = 3
batches_per_epoch = len(dict_dataset["train"]) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)

optimizer, schedule = create_optimizer(
    init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps
)
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


El aviso se "queja" que no hemos proporcionado ninguna función de error para entrenar el modelo. En este caso, por defecto, se utilizará la función loss. 

Sin embargo, es habitual que queramos entrenar y ajustar nuestro modelo en función a otras métricas distintas a loss. Podríamos utilizar accuracy, pero tampoco esta métrica no es muy adecuada cuando estamos trabajando en un problema de multiclasificación de 6 clases. Por tanto, siempre es interesante informar sobre otras métricas como precisión, recall y F1 en su versión macro. La macro precisión es la media de todas las precisiones entre el número de clases. De forma similar, se define la macro recall y macro F1.

Además de la función que computa estas métricas, necesitamos crear un objeto de tipo **KerasMetricCallback**, que vamos poder pasarle al modelo de tensorflow para indicarle la función de evaluación y sobre que conjunto:

In [ ]:
from transformers.keras_callbacks import KerasMetricCallback
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np


def compute_metrics(eval_predictions):
    predictions, labels = eval_predictions
    #     predictions = np.argmax(predictions, axis=1)
    y_pred = predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, y_pred, average='macro')
    acc = accuracy_score(labels, y_pred)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

metric_callback = KerasMetricCallback(
    metric_fn=compute_metrics, eval_dataset=tf_validation_dataset
)


Ahora si ya podemos entrenar el modelo, porque además de indicarle, el conjunto de training y validación, y el número de epochs, también indicamos en el parámetro callbacks, los detalles de la evaluación. Vamos a entrenar:

In [ ]:

model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
    epochs=num_epochs,
    callbacks=[metric_callback],
)

Epoch 1/3
306/306 [==============================] - 71s 102ms/step - loss: 0.7182 - val_loss: 0.2266 - accuracy: 0.9377 - f1: 0.7898 - precision: 0.7872 - recall: 0.7926
Epoch 2/3
  1/306 [..............................] - ETA: 15s - loss: 0.2111

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


306/306 [==============================] - 15s 48ms/step - loss: 0.1925 - val_loss: 0.1899 - accuracy: 0.9432 - f1: 0.9115 - precision: 0.9239 - recall: 0.9014
Epoch 3/3
306/306 [==============================] - 15s 49ms/step - loss: 0.1063 - val_loss: 0.1896 - accuracy: 0.9432 - f1: 0.9118 - precision: 0.9222 - recall: 0.9037


## Evaluation 
Evalúamos el modelos sobre los textos del conjunto test:

In [ ]:
def get_prediction(text):
    # preparamos el texto igual que lo hicimos para los de training
    inputs = tokenizer(text, padding="max_length", truncation=True, max_length=MAX_LENGTH, return_tensors="np")
    outputs = model(inputs).logits # la salida está en la propiedada logists, que es un numpy array
    # con las probabilidades de cada clase. Debo tomar la mayor.
    return np.argmax(outputs, axis=1)


Predecimos las salidas para todos los textos del test y guardamos en y_pred:

In [ ]:
y_pred=[get_prediction(text) for text in dict_dataset['test']['text']]
print(y_pred[:5])


[array([5]), array([4]), array([3]), array([2]), array([5])]


Podemos utilizar la función proporcionada por sklearn **classification_report** para obtener los resultados finales

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true=dict_dataset['test']['label'], y_pred=y_pred, target_names=TARGET_LABELS))

              precision    recall  f1-score   support

        ABBR       1.00      0.78      0.88         9
        ENTY       0.99      0.87      0.93        94
        DESC       0.96      1.00      0.98       138
         HUM       0.96      0.98      0.97        65
         LOC       0.96      0.98      0.97        81
         NUM       0.97      1.00      0.98       113

    accuracy                           0.97       500
   macro avg       0.97      0.94      0.95       500
weighted avg       0.97      0.97      0.97       500



Source: https://github.com/huggingface/notebooks/blob/main/examples/text_classification-tf.ipynb
